## This script accesses the CESM2 Discharge data for the Historical years from GLADE repository of NCAR

In [1]:
import xarray as xr
from netCDF4 import Dataset  
import numpy as np
from numpy import unravel_index

from matplotlib import pyplot as plt 
from numpy import linspace 

In [2]:
# Projection
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [3]:
from descartes import PolygonPatch
import shapefile as shp  # Requires the pyshp package
import cartopy.io.shapereader as shpreader

In [4]:
# Common imports
import calendar
import datetime
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from numpy import array, ma
import scipy.io

In [5]:
cd '/glade/campaign/cgd/cesm/CESM2-LE/rof/proc/tseries/month_1'

/glade/campaign/cgd/cesm/CESM2-LE/rof/proc/tseries/month_1


In [7]:
cd RIVER_DISCHARGE_OVER_LAND_LIQ

/glade/campaign/cgd/cesm/CESM2-LE/rof/proc/tseries/month_1/RIVER_DISCHARGE_OVER_LAND_LIQ


In [8]:
pwd

'/glade/campaign/cgd/cesm/CESM2-LE/rof/proc/tseries/month_1/RIVER_DISCHARGE_OVER_LAND_LIQ'

## Accessing One File

In [9]:
filePath = '/glade/campaign/cgd/cesm/CESM2-LE/rof/proc/tseries/month_1/RIVER_DISCHARGE_OVER_LAND_LIQ'
fileName_Sample = 'b.e21.BHISTcmip6.f09_g17.LE2-1001.001.mosart.h0.RIVER_DISCHARGE_OVER_LAND_LIQ.185001-185912.nc'
ncFile_disch_Sample = str(filePath) + '/' + str(fileName_Sample)
print(ncFile_disch_Sample)

/glade/campaign/cgd/cesm/CESM2-LE/rof/proc/tseries/month_1/RIVER_DISCHARGE_OVER_LAND_LIQ/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.mosart.h0.RIVER_DISCHARGE_OVER_LAND_LIQ.185001-185912.nc


In [10]:
file_xr_Sample  = xr.open_dataset(ncFile_disch_Sample) 
file_xr_Sample

<xarray.Dataset>
Dimensions:                        (hist_interval: 2, lat: 360, lon: 720, time: 120)
Coordinates:
  * lon                            (lon) float64 -179.8 -179.2 ... 179.2 179.8
  * lat                            (lat) float64 -89.75 -89.25 ... 89.25 89.75
  * time                           (time) datetime64[ns] 1850-02-01 ... 1860-01-01
Dimensions without coordinates: hist_interval
Data variables:
    mask                           (lat, lon) float64 ...
    area                           (lat, lon) float64 ...
    areatotal                      (lat, lon) float64 ...
    areatotal2                     (lat, lon) float64 ...
    mcdate                         (time) float64 ...
    mcsec                          (time) float64 ...
    mdcur                          (time) float64 ...
    mscur                          (time) float64 ...
    nstep                          (time) float64 ...
    time_bounds                    (time, hist_interval) float64 ...
    date_wr

In [11]:
file_disch_Sample = Dataset(ncFile_disch_Sample, mode='r')
file_disch_Sample

lons = file_disch_Sample.variables['lon'][:]
lats = file_disch_Sample.variables['lat'][:]
times = file_disch_Sample.variables['time'][:]
ntim = times.shape[0]
print(ntim)

time = np.arange(ntim)
time

param_disch_Sample = file_disch_Sample.variables['RIVER_DISCHARGE_OVER_LAND_LIQ'][:]

print(param_disch_Sample.shape)
print('')

120
(120, 360, 720)



In [12]:
t,y,x = param_disch_Sample.shape
print(y)
print(x)
print(t)

nDecades = 16
monStepHistTot = t*nDecades + 60   # =185001_201412 (60 is half of 120)
print(monStepHistTot)

ensMem = 50     # 50 ensemble members for monthly discharge data
print(ensMem)

360
720
120
1980
50


## Setting US boundaries

In [13]:
# Save only U.S. 
latboundsUs = [ 25 , 55]
lonboundsUs = [-130 , -60]


In [14]:
# latitude lower and upper index
latli = np.argmin( np.abs( lats - latboundsUs[0] ) )
latui = np.argmin( np.abs( lats - latboundsUs[1] ) ) 

# longitude lower and upper index
lonli = np.argmin( np.abs( lons - lonboundsUs[0] ) )
lonui = np.argmin( np.abs( lons - lonboundsUs[1] ) )  

In [15]:
print(param_disch_Sample.shape)

(120, 360, 720)


In [16]:
param_disch_Sample_US = param_disch_Sample[:, latli:latui , lonli:lonui]
print(param_disch_Sample_US.shape)

(120, 60, 140)


In [17]:
t,y_US,x_US = param_disch_Sample_US.shape
print(y_US)
print(x_US)
print(t)

60
140
120


In [18]:
#
monDischUsHistEnsMem = np.zeros((monStepHistTot, y_US, x_US, ensMem, )) 
monDischUsHistEnsMem[:] = np.nan
print(monDischUsHistEnsMem.shape)

(1980, 60, 140, 50)


In [19]:

filePath = '/glade/campaign/cgd/cesm/CESM2-LE/rof/proc/tseries/month_1/RIVER_DISCHARGE_OVER_LAND_LIQ'

yrCtrlIds = ['1001', '1021', '1041', '1061', '1081', '1101', '1121', '1141', '1161', '1181', '1231', '1251', '1281', '1301'];
yrId = 0
monId = 0

monSt = 1
monEnd = 12

memId = -1
for yrCtrl in yrCtrlIds:    
    yrId = yrId + 1   
    
    if yrId < 11:
        memId = memId + 1
        tStepSt = 0
        yrHistSt = 1849
        yrHistEnd = yrHistSt + 9
    
        monId = monId + 1  
        yrMonDigits = str(yrCtrl)+'.' + str(monId).rjust(3,'0')
        print(yrMonDigits)
        
        for decade in range(17):
            yrHistSt = yrHistSt + 1
            yrHistEnd = yrHistEnd + 1
            
            yrMonHistSt = str(yrHistSt) + str(monSt).rjust(2,'0')
            yrMonHistEnd = str(yrHistEnd) + str(monEnd).rjust(2,'0')  
            
            fileName = 'b.e21.BHISTcmip6.f09_g17.LE2-'+yrMonDigits+'.mosart.h0.RIVER_DISCHARGE_OVER_LAND_LIQ.' +yrMonHistSt+'-'+yrMonHistEnd+'.nc'
            ncFile_disch = str(filePath) + '/' + str(fileName)
            file_disch = Dataset(ncFile_disch, mode='r')                        
            param_DischToAdd = file_disch.variables['RIVER_DISCHARGE_OVER_LAND_LIQ'][:, latli:latui , lonli:lonui]            
            t,y_US,x_US = param_DischToAdd.shape
            tStepEnd = tStepSt + t            
           
            
            monDischUsHistEnsMem[tStepSt:tStepEnd, :, :, memId] = param_DischToAdd  
            
            yrHistSt = yrHistEnd
            tStepSt = tStepEnd
            
            if decade <15:
                yrHistEnd = yrHistSt + 9
            else:
                yrHistEnd = yrHistSt + 4
            
    else: 
        
    
        for monCont in range(1, 11): 
            
            tStepSt = 0
            memId = memId + 1
            yrMonDigits = str(yrCtrl)+'.' + str(monCont).rjust(3,'0')
            print(yrMonDigits)
            
            yrHistSt = 1849
            yrHistEnd = yrHistSt + 9
            
            for decade in range(17):            
                yrHistSt = yrHistSt + 1
                yrHistEnd = yrHistEnd + 1
    
                yrMonHistSt = str(yrHistSt) + str(monSt).rjust(2,'0')
                yrMonHistEnd = str(yrHistEnd) + str(monEnd).rjust(2,'0')  
                
                fileName = 'b.e21.BHISTcmip6.f09_g17.LE2-'+yrMonDigits+'.mosart.h0.RIVER_DISCHARGE_OVER_LAND_LIQ.' +yrMonHistSt+'-'+yrMonHistEnd+'.nc'
                ncFile_disch = str(filePath) + '/' + str(fileName)
                file_disch = Dataset(ncFile_disch, mode='r')                        
                param_DischToAdd = file_disch.variables['RIVER_DISCHARGE_OVER_LAND_LIQ'][:, latli:latui , lonli:lonui]              
                t,y_US,x_US = param_DischToAdd.shape
                tStepEnd = tStepSt + t 
                
                    
                monDischUsHistEnsMem[tStepSt:tStepEnd, :, :, memId] = param_DischToAdd 
                
            
                yrHistSt = yrHistEnd
                tStepSt = tStepEnd               
                
                if decade <15:
                    yrHistEnd = yrHistSt + 9
                else:
                    yrHistEnd = yrHistSt + 4
        
    


1001.001
1021.002
1041.003
1061.004
1081.005
1101.006
1121.007
1141.008
1161.009
1181.010
1231.001
1231.002
1231.003
1231.004
1231.005
1231.006
1231.007
1231.008
1231.009
1231.010
1251.001
1251.002
1251.003
1251.004
1251.005
1251.006
1251.007
1251.008
1251.009
1251.010
1281.001
1281.002
1281.003
1281.004
1281.005
1281.006
1281.007
1281.008
1281.009
1281.010
1301.001
1301.002
1301.003
1301.004
1301.005
1301.006
1301.007
1301.008
1301.009
1301.010


In [20]:
print(monDischUsHistEnsMem.shape)

(1980, 60, 140, 50)


## Subsetting data for 1930~2014 as required for GPC paper

In [21]:
print(monDischUsHistEnsMem.shape)

(1980, 60, 140, 50)


In [22]:
histYrSt  = 1850
histTrgtYrSt = 1930
histYrEnd = 2014

nYrsHist = (histYrEnd - histTrgtYrSt + 1)
print(nYrsHist)

85


In [23]:
histTrgtStepSt = (histTrgtYrSt - histYrSt)*12     # = from 1930
histTrgtStepEnd = (histYrEnd - histYrSt + 1)*12
print(histTrgtStepSt, histTrgtStepEnd)


960 1980


In [24]:
print(monDischUsHistEnsMem.shape)

(1980, 60, 140, 50)


In [25]:
monDischUsHistSelEnsMembers = monDischUsHistEnsMem[histTrgtStepSt:histTrgtStepEnd,:,:,:]
print(monDischUsHistSelEnsMembers.shape)

(1020, 60, 140, 50)


## Calculate Ensemble Mean value

In [27]:
monDischUsHistSelEnsMean = np.nanmean(monDischUsHistSelEnsMembers, axis = 3)
print(monDischUsHistSelEnsMean.shape)

(1020, 60, 140)


In [45]:
np.save('/glade/scratch/mrhaider/cesm2/param/discharge/mon/monDischUsHistSelEnsMean.npy', monDischUsHistSelEnsMean)

In [36]:
lats_US = lats[latli:latui]
lons_US = lons[lonli:lonui]

print(lats_US)


[24.75 25.25 25.75 26.25 26.75 27.25 27.75 28.25 28.75 29.25 29.75 30.25
 30.75 31.25 31.75 32.25 32.75 33.25 33.75 34.25 34.75 35.25 35.75 36.25
 36.75 37.25 37.75 38.25 38.75 39.25 39.75 40.25 40.75 41.25 41.75 42.25
 42.75 43.25 43.75 44.25 44.75 45.25 45.75 46.25 46.75 47.25 47.75 48.25
 48.75 49.25 49.75 50.25 50.75 51.25 51.75 52.25 52.75 53.25 53.75 54.25]


In [37]:
latsUsArr = np.array(lats_US)
print(latsUsArr)

lonsUsArr = np.array(lons_US)
print(lonsUsArr)


[24.75 25.25 25.75 26.25 26.75 27.25 27.75 28.25 28.75 29.25 29.75 30.25
 30.75 31.25 31.75 32.25 32.75 33.25 33.75 34.25 34.75 35.25 35.75 36.25
 36.75 37.25 37.75 38.25 38.75 39.25 39.75 40.25 40.75 41.25 41.75 42.25
 42.75 43.25 43.75 44.25 44.75 45.25 45.75 46.25 46.75 47.25 47.75 48.25
 48.75 49.25 49.75 50.25 50.75 51.25 51.75 52.25 52.75 53.25 53.75 54.25]
[-130.25 -129.75 -129.25 -128.75 -128.25 -127.75 -127.25 -126.75 -126.25
 -125.75 -125.25 -124.75 -124.25 -123.75 -123.25 -122.75 -122.25 -121.75
 -121.25 -120.75 -120.25 -119.75 -119.25 -118.75 -118.25 -117.75 -117.25
 -116.75 -116.25 -115.75 -115.25 -114.75 -114.25 -113.75 -113.25 -112.75
 -112.25 -111.75 -111.25 -110.75 -110.25 -109.75 -109.25 -108.75 -108.25
 -107.75 -107.25 -106.75 -106.25 -105.75 -105.25 -104.75 -104.25 -103.75
 -103.25 -102.75 -102.25 -101.75 -101.25 -100.75 -100.25  -99.75  -99.25
  -98.75  -98.25  -97.75  -97.25  -96.75  -96.25  -95.75  -95.25  -94.75
  -94.25  -93.75  -93.25  -92.75  -92.25  -91.75  

In [32]:
np.save('/glade/scratch/mrhaider/cesm2/param/discharge/mon/latsUsArr.npy', latsUsArr)

In [33]:
np.save('/glade/scratch/mrhaider/cesm2/param/discharge/mon/lonsUsArr.npy', lonsUsArr)

In [44]:
STOP here ===========

SyntaxError: invalid syntax (<ipython-input-44-665768e7338e>, line 1)